# ２章　小売店のデータでデータ加工を行う１０本ノック

本章では、ある小売店の売上履歴と顧客台帳データを用いて、データ分析の素地となる「データの加工」を習得することが目的です。
実際の現場データは手入力のExcel等、決して綺麗なデータではない事が多いため、
データの揺れや整合性の担保など、汚いデータを取り扱うデータ加工を主体に進めて行きます。

### ノック１１：データを読み込んでみよう

In [1]:
import pandas as pd
uriage_data = pd.read_csv('uriage.csv')
uriage_data.head()

,purchase_date,item_name,item_price,customer_name
0,2019-06-13 18:02:34,商品A,100.0,深井菜々美
1,2019-07-13 13:05:29,商 品 S,NaN,浅田賢二
2,2019-05-11 19:42:07,商 品 a,NaN,南部慶二
3,2019-02-12 23:40:45,商品Z,2600.0,麻生莉緒
4,2019-04-22 03:09:35,商品a,NaN,平田鉄二


In [15]:
kokyaku_data = pd.read_excel('kokyaku_daicho.xlsx')
kokyaku_data.head()

,顧客名,かな,地域,メールアドレス,登録日
0,須賀ひとみ,すが ひとみ,H市,suga_hitomi@example.com,2018/01/04
1,岡田 敏也,おかだ としや,E市,okada_toshiya@example.com,42782
2,芳賀 希,はが のぞみ,A市,haga_nozomi@example.com,2018/01/07
3,荻野 愛,おぎの あい,F市,ogino_ai@example.com,42872
4,栗田 憲一,くりた けんいち,E市,kurita_kenichi@example.com,43127


### ノック１２：データの揺れを見てみよう

In [16]:
uriage_data['item_name'].head()

0      商品A
1    商 品 S
2    商 品 a
3      商品Z
4      商品a
Name: item_name, dtype: object

In [17]:
uriage_data['item_price'].head()

0     100.0
1       NaN
2       NaN
3    2600.0
4       NaN
Name: item_price, dtype: float64

### ノック１３：データに揺れがあるまま集計しよう

In [21]:
uriage_data['purchase_date'] = pd.to_datetime(uriage_data['purchase_date'])
uriage_data['purchase_month'] = uriage_data['purchase_date'].dt.strftime("%Y%m")
res = uriage_data.pivot_table(index='purchase_month', columns='item_name', aggfunc='size', fill_value=0)
res

item_name,商品W,商 品 n,商品E,商品M,商品P,商品S,商品W,商品X,商 品O,商 品Q,...,商品k,商品l,商品o,商品p,商品r,商品s,商品t,商品v,商品x,商品y
purchase_month,,,,,,,,,,,,,,,,,,,,,
201901,0,1,0,0,0,0,0,0,0,0,...,1,1,1,0,0,0,0,0,0,0
201902,0,0,0,0,0,0,0,1,0,0,...,0,0,0,0,0,1,1,1,0,0
201903,0,0,1,1,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
201904,1,0,0,0,0,0,0,0,0,1,...,0,0,0,0,0,1,0,0,0,0
201905,0,0,0,0,0,1,0,0,0,0,...,0,1,0,0,0,0,0,0,0,1
201906,0,0,0,0,0,0,1,0,0,0,...,0,0,0,1,0,0,0,0,1,0
201907,0,0,0,0,0,0,0,0,1,0,...,0,0,1,0,2,0,0,0,0,0


In [24]:
res = uriage_data.pivot_table(index='purchase_month', columns='item_name', values='item_price', aggfunc='sum', fill_value=0)
res

item_name,商品W,商 品 n,商品E,商品M,商品P,商品S,商品W,商品X,商 品O,商 品Q,...,商品k,商品l,商品o,商品p,商品r,商品s,商品t,商品v,商品x,商品y
purchase_month,,,,,,,,,,,,,,,,,,,,,
201901,0,1400,0,0,0,0,0,0,0,0,...,1100,1200,1500,0,0,0,0,0,0,0
201902,0,0,0,0,0,0,0,2400,0,0,...,0,0,0,0,0,1900,2000,2200,0,0
201903,0,0,500,1300,1600,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
201904,2300,0,0,0,0,0,0,0,0,1700,...,0,0,0,0,0,1900,0,0,0,0
201905,0,0,0,0,0,1900,0,0,0,0,...,0,1200,0,0,0,0,0,0,0,2500
201906,0,0,0,0,0,0,2300,0,0,0,...,0,0,0,1600,0,0,0,0,2400,0
201907,0,0,0,0,0,0,0,0,0,0,...,0,0,1500,0,1800,0,0,0,0,0


### ノック１４：商品名の揺れを補正しよう

In [27]:
print(len(pd.unique(uriage_data.item_name)))

99


In [36]:
uriage_data['item_name'] = uriage_data['item_name'].str.upper()
uriage_data['item_name'] = uriage_data['item_name'].str.replace('　', '')
uriage_data['item_name'] = uriage_data['item_name'].str.replace(' ', '')
uriage_data.sort_values(by=['item_name'], ascending=True)

,purchase_date,item_name,item_price,customer_name,purchase_month
0,2019-06-13 18:02:34,商品A,100.0,深井菜々美,201906
1748,2019-05-19 20:22:22,商品A,100.0,松川綾女,201905
223,2019-06-25 08:13:20,商品A,100.0,板橋隆,201906
1742,2019-06-13 16:03:17,商品A,100.0,小平陽子,201906
1738,2019-02-10 00:28:43,商品A,100.0,松田浩正,201902
...,...,...,...,...,...
2880,2019-04-22 00:36:52,商品Y,NaN,田辺光洋,201904
2881,2019-04-30 14:21:09,商品Y,NaN,高原充則,201904
1525,2019-01-24 10:27:23,商品Y,2500.0,五十嵐春樹,201901
1361,2019-05-28 13:45:32,商品Y,2500.0,大崎ヒカル,201905


In [35]:
print(pd.unique(uriage_data.item_name))
print(len(pd.unique(uriage_data.item_name)))

['商品A' '商品S' '商品Z' '商品V' '商品O' '商品U' '商品L' '商品C' '商品I' '商品R' '商品X' '商品G'
 '商品P' '商品Q' '商品Y' '商品N' '商品W' '商品E' '商品K' '商品B' '商品F' '商品D' '商品M' '商品H'
 '商品T' '商品J']
26


### ノック１５：金額欠損値の補完をしよう

In [38]:
uriage_data.isnull().any(axis=0)

purchase_date     False
item_name         False
item_price         True
customer_name     False
purchase_month    False
dtype: bool

In [61]:
flg_is_null = uriage_data['item_price'].isnull()
print(list(uriage_data.loc[flg_is_null, 'item_name'].unique()))
for trg in list(uriage_data.loc[flg_is_null, 'item_name'].unique()):
    price = uriage_data.loc[(~flg_is_null) & (uriage_data['item_name'] == trg), 'item_price'].max()
    uriage_data.loc[flg_is_null & (uriage_data['item_name'] == trg), 'item_price'] = price
uriage_data.head()

[]


,purchase_date,item_name,item_price,customer_name,purchase_month
0,2019-06-13 18:02:34,商品A,100.0,深井菜々美,201906
1,2019-07-13 13:05:29,商品S,1900.0,浅田賢二,201907
2,2019-05-11 19:42:07,商品A,100.0,南部慶二,201905
3,2019-02-12 23:40:45,商品Z,2600.0,麻生莉緒,201902
4,2019-04-22 03:09:35,商品A,100.0,平田鉄二,201904


In [64]:
uriage_data.isnull().any(axis=0)

purchase_date     False
item_name         False
item_price        False
customer_name     False
purchase_month    False
dtype: bool

In [72]:
for trg in list(uriage_data['item_name'].sort_values().unique()):
    print(trg + 'の最大額 == 最少額: ' + str((uriage_data.loc[uriage_data['item_name'] == trg]['item_price'].max()) == (uriage_data.loc[uriage_data['item_name'] == trg]['item_price'].min(skipna=False))))


商品Aの最大額 == 最少額: True
商品Bの最大額 == 最少額: True
商品Cの最大額 == 最少額: True
商品Dの最大額 == 最少額: True
商品Eの最大額 == 最少額: True
商品Fの最大額 == 最少額: True
商品Gの最大額 == 最少額: True
商品Hの最大額 == 最少額: True
商品Iの最大額 == 最少額: True
商品Jの最大額 == 最少額: True
商品Kの最大額 == 最少額: True
商品Lの最大額 == 最少額: True
商品Mの最大額 == 最少額: True
商品Nの最大額 == 最少額: True
商品Oの最大額 == 最少額: True
商品Pの最大額 == 最少額: True
商品Qの最大額 == 最少額: True
商品Rの最大額 == 最少額: True
商品Sの最大額 == 最少額: True
商品Tの最大額 == 最少額: True
商品Uの最大額 == 最少額: True
商品Vの最大額 == 最少額: True
商品Wの最大額 == 最少額: True
商品Xの最大額 == 最少額: True
商品Yの最大額 == 最少額: True
商品Zの最大額 == 最少額: True


### ノック１６：顧客名の揺れを補正しよう

In [74]:
kokyaku_data['顧客名'].head()

0     須賀ひとみ
1    岡田　 敏也
2      芳賀 希
3     荻野  愛
4     栗田 憲一
Name: 顧客名, dtype: object

In [75]:
uriage_data['customer_name'].head()

0    深井菜々美
1     浅田賢二
2     南部慶二
3     麻生莉緒
4     平田鉄二
Name: customer_name, dtype: object

In [84]:
kokyaku_data['顧客名'] = kokyaku_data['顧客名'].str.replace('　', "")
kokyaku_data['顧客名'] = kokyaku_data['顧客名'].str.replace(' ', "")
pd.unique(kokyaku_data['顧客名'])

array(['須賀ひとみ', '岡田敏也', '芳賀希', '荻野愛', '栗田憲一', '梅沢麻緒', '相原ひとり', '新村丈史',
       '石川まさみ', '小栗正義', '大倉晃司', '那須蒼甫', '清水裕次郎', '榊原しぼり', '高沢美咲', '川島友以乃',
       '唐沢景子', '稲田将也', '秋葉あき', '西脇礼子', '内村まさみ', '進藤瞬', '小口豊', '笹原しぼり',
       '岩佐孝太郎', '河村由樹', '菅原誠治', '榎本薫', '高梨結衣', '鶴岡薫', '藤木一恵', '森岡季衣',
       '田畑正敏', '宇野秀樹', '高原充則', '梅村秀樹', '浅田賢二', '石渡小雁', '吉岡サダヲ', '井口寛治',
       '奥光洋', '佐藤慶二', '東光博', '高田さんま', '合田光', '深田信輔', '島英嗣', '梅津淳子',
       '日比野徹', '熊倉綾', '松田浩正', '楠高史', '岡慶太', '市田寿明', '山西花', '青山鉄洋', '松川綾女',
       '大滝麗奈', '赤木愛梨', '楠哲平', '堀内聡', '八木雅彦', '林勇', '日野夏希', '麻生莉緒', '根岸仁晶',
       '赤木だん吉', '亀井一徳', '横田遥', '野本仁晶', '西原未華子', '五十嵐春樹', '堀江佑', '久保田倫子',
       '金森なつみ', '城戸芳正', '望月真悠子', '井本マサカズ', '米沢仁晶', '本橋直人', '古川信吾', '岩沢那奈',
       '西島知世', '戸塚美幸', '深井照生', '新谷智花', '有馬徹平', '本多フミヤ', '浜田未華子', '牧田玲那',
       '島孝太郎', '石田花', '星野美嘉', '黒谷長利', '深沢ひろ子', '矢部夏空', '立石茜', '手塚雅之',
       '松谷愛子', '中田美智子', '片瀬長利', '石崎幸子', '明石家明', '小川美菜', '田辺光洋', '神原美嘉',
       '岩城徹平', '相川良介', '植村遥', '井川真悠子', '松元翔太', 'さだ千佳子', '島本研

### ノック１７：日付の揺れを補正しよう

In [92]:
print(kokyaku_data.head())

     顧客名        かな  地域                     メールアドレス         登録日
0  須賀ひとみ    すが ひとみ  H市     suga_hitomi@example.com  2018/01/04
1   岡田敏也   おかだ としや  E市   okada_toshiya@example.com       42782
2    芳賀希    はが のぞみ  A市     haga_nozomi@example.com  2018/01/07
3    荻野愛    おぎの あい  F市        ogino_ai@example.com       42872
4   栗田憲一  くりた けんいち  E市  kurita_kenichi@example.com       43127


In [93]:
flg_is_serial = kokyaku_data['登録日'].astype(str).str.isdigit()
flg_is_serial.sum()

22

In [97]:
print(kokyaku_data.loc[flg_is_serial, '登録日'])
fromSerial = pd.to_timedelta(kokyaku_data.loc[flg_is_serial, '登録日'].astype('float'), unit='D') + pd.to_datetime('1900/01/01')
fromSerial

1      42782
3      42872
4      43127
21     42920
27     42901
47     42741
49     42929
53     42833
76     43188
80     43110
99     42885
114    43254
118    43129
122    43206
139    42880
143    42818
155    42754
172    43181
179    42743
183    42940
186    43294
192    43259
Name: 登録日, dtype: object


1     2017-02-18
3     2017-05-19
4     2018-01-29
21    2017-07-06
27    2017-06-17
47    2017-01-08
49    2017-07-15
53    2017-04-10
76    2018-03-31
80    2018-01-12
99    2017-06-01
114   2018-06-05
118   2018-01-31
122   2018-04-18
139   2017-05-27
143   2017-03-26
155   2017-01-21
172   2018-03-24
179   2017-01-10
183   2017-07-26
186   2018-07-15
192   2018-06-10
Name: 登録日, dtype: datetime64[ns]

In [100]:
fromString = pd.to_datetime(kokyaku_data.loc[~flg_is_serial, '登録日'])
fromString

0     2018-01-04
2     2018-01-07
5     2017-06-20
6     2018-06-11
7     2017-05-19
         ...    
195   2017-06-20
196   2018-06-20
197   2017-04-29
198   2019-04-19
199   2019-04-23
Name: 登録日, Length: 178, dtype: datetime64[ns]

In [103]:
kokyaku_data['登録日'] = pd.concat([fromSerial, fromString])
kokyaku_data['登録日']

0     2018-01-04
1     2017-02-18
2     2018-01-07
3     2017-05-19
4     2018-01-29
         ...    
195   2017-06-20
196   2018-06-20
197   2017-04-29
198   2019-04-19
199   2019-04-23
Name: 登録日, Length: 200, dtype: datetime64[ns]

In [113]:
kokyaku_data['登録年月'] = kokyaku_data['登録日'].dt.strftime('%Y%m')
rslt = kokyaku_data.groupby('登録年月').count()['顧客名']
print(rslt)
print(len(kokyaku_data))

登録年月
201701    15
201702    11
201703    14
201704    15
201705    13
201706    14
201707    17
201801    13
201802    15
201803    17
201804     5
201805    19
201806    13
201807    17
201904     2
Name: 顧客名, dtype: int64
200


In [117]:
flg_is_serial = kokyaku_data['登録日'].astype('str').str.isdigit()
flg_is_serial.sum()

0

### ノック１８：顧客名をキーに２つのデータを結合(ジョイン)しよう

In [121]:
joined_data = pd.merge(uriage_data, kokyaku_data, left_on='customer_name', right_on='顧客名', how='left')
joined_data = joined_data.drop('customer_name', axis=1)
joined_data.head()

,purchase_date,item_name,item_price,purchase_month,顧客名,かな,地域,メールアドレス,登録日,登録年月
0,2019-06-13 18:02:34,商品A,100.0,201906,深井菜々美,ふかい ななみ,C市,fukai_nanami@example.com,2017-01-26,201701
1,2019-07-13 13:05:29,商品S,1900.0,201907,浅田賢二,あさだ けんじ,C市,asada_kenji@example.com,2018-04-07,201804
2,2019-05-11 19:42:07,商品A,100.0,201905,南部慶二,なんぶ けいじ,A市,nannbu_keiji@example.com,2018-06-19,201806
3,2019-02-12 23:40:45,商品Z,2600.0,201902,麻生莉緒,あそう りお,D市,asou_rio@example.com,2018-07-22,201807
4,2019-04-22 03:09:35,商品A,100.0,201904,平田鉄二,ひらた てつじ,D市,hirata_tetsuji@example.com,2017-06-07,201706


### ノック１９：クレンジングしたデータをダンプしよう

### ノック２０：データを集計しよう